1. Read File,
2. Prep Data for single feature
3. Combine prep data
4. dump to oven ready

Tasks in each feature extraction
in : data frame
out: processed dataframe

1. add some setup of columns
2. group by some aggregations, may be
3. drop if no longer used
4. gc collect
5  write dataframe to csv
5. return the processed data frame, and csv name


In [1]:
import pandas as pd
import os

In [2]:
train_file='eda/input/train_specific_hours_sample.csv'
# valid_file='eda/input/valid_specific_hours_sample.csv'
valid_file=('eda/input/valid_specific_hours_sampletest_hours.csv')
test_file='input/test_parsed.csv'
comment="_valid_hr_is_test_hr"
overwrite=True
file_in_progress=None

In [3]:
def set_file(name):
    global file_in_progress
    file_in_progress=name
def feature_file(file):
    return str.lower(folder_path(file)+csv_name(file))
def csv_name(file):
    if 'csv' not in file:
        return file+comment+'.csv' 
    return file
def folder_path(name):
    folder=get_folder(name)
    if not os.path.exists(folder):
        os.makedirs(folder)
    return folder
def get_folder(name):
    global file_in_progress
    if 'train' in file_in_progress:
        return 'eda/features/'+'train/'
    elif 'valid' in file_in_progress:
        return 'eda/features/'+'valid/'
    elif 'test' in file_in_progress:
        return 'eda/features/'+'test/'
    else:
        return 'eda/features/'
    
def log_feature(df,name,group_cols):
    if name is 'base_features':
        df.to_csv(feature_file(name),index=False,header=True)
    else:
        df[list(set().union([name],group_cols))].head()
        df[list(set().union([name],group_cols))].to_csv(feature_file(name),index=False,header=True)

def skip_if_already_exists(agg_name):
    global overwrite
    if not overwrite:
        if os.path.exists(feature_file(agg_name)):
            print(feature_file(agg_name) + ' already exists')
            df = pd.read_csv(feature_file(agg_name))
            return df
        else:
            return None
    else:
        if os.path.exists(feature_file(agg_name)):
            print('deleting ',feature_file(agg_name),'...')
            os.remove(feature_file(agg_name))
        return None

In [4]:
IN_TEST_HH='IN_TEST_HH'
def in_test_hh(df,key=IN_TEST_HH):
    gp = skip_if_already_exists(key)
    if gp is not None:
        df[key]=gp[key]
        return df
    else:
        most_freq_hours_in_test_data = [4, 5, 9, 10, 13, 14]
        least_freq_hours_in_test_data = [6, 11, 15]
        import gc
        gc.collect()
        df[key] = (3 
                - 2*df['hour'].isin(most_freq_hours_in_test_data ) 
                - 1*df['hour'].isin( least_freq_hours_in_test_data ) ).astype('uint8')
        log_feature(df,key,[])
        return df

In [5]:
def aggregate_cols( df, group_cols, agg_name, agg_type='uint32', show_max=False, show_agg=True):
    if show_agg:
        print( "Aggregating by ", group_cols , '...' )
    gp = skip_if_already_exists(agg_name) 
    if gp is None:
        gp = df[group_cols][group_cols].groupby(group_cols).size().rename(agg_name).to_frame().reset_index()
        log_feature(gp,agg_name,group_cols)
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    import gc
    gc.collect()
    return df

In [6]:
BASE_FEATURES='base_features'
def base_features(df,key=BASE_FEATURES):
    gp=skip_if_already_exists(key)
    if gp is None:
        log_feature(df,key,group_cols=[])
        return df
    return gp

In [7]:
#GROUP FEATURES
IP_DAY_TEST_HH='IP_DAY_TEST_HH'
IP_DAY_HH='IP_DAY_HH'
IP_OS_HH='IP_OS_HH'
IP_APP_HH='IP_APP_HH'
IP_APP_OS_HH='IP_APP_OS_HH'
APP_DAY_HH='APP_DAY_HH'
def ip_day_test(df,key = IP_DAY_TEST_HH):
    return aggregate_cols(df, ['ip', 'day', IN_TEST_HH], key, show_max=True )
def ip_day_hh(df,key = IP_DAY_HH):
    return aggregate_cols( df, ['ip', 'day', 'hour'], key, 'uint16', show_max=True );
def ip_os_hh(df,key = IP_OS_HH):
    return aggregate_cols( df, ['ip', 'day', 'os', 'hour'], key, 'uint16', show_max=True );
def ip_app_hh(df,key = IP_APP_HH):
    return aggregate_cols( df, ['ip', 'day', 'app', 'hour'], key, 'uint16', show_max=True );
def ip_app_os_hh(df,key = IP_APP_OS_HH):
    return aggregate_cols( df, ['ip', 'day', 'app', 'os', 'hour'], key, 'uint16', show_max=True );
def app_day_hh(df,key = APP_DAY_HH):
    return aggregate_cols( df, ['app', 'day', 'hour'], key, 'uint16', show_max=True );

In [8]:
def joint_name(cols,prefix=''):
    return prefix+'_'.join(cols).replace('is_attributed_','')[0:80]
def cluster(df,n):
    res=AgglomerativeClustering(n_clusters=n).fit(df)
    import pandas as pd 
    df_clusters = pd.DataFrame(res.labels_)
    df_clusters.to_csv(joint_name(df.columns),index=False)
    return (df_clusters)

In [9]:
NON_FRAUD_PERC_IP='NON_FRAUD_PERC_IP'
def get_non_fraud_perc_ip(df,col=['ip']):
    if 'is_attributed' not in df.columns:
        print('is attriuted not a part of columns, skipping this one')
        return df
    agg_name=joint_name(col,'perc_')
    gp = skip_if_already_exists(agg_name) 
    if gp is None:
        val=df.groupby(col)['is_attributed']
        gp=(val.sum()/val.count()).rename(agg_name).to_frame().reset_index()
        gp=gp.reset_index()
        log_feature(gp,name=agg_name,group_cols=col)
    df[agg_name] = gp[agg_name]
    del gp
    return df

In [10]:
feature_config={
    BASE_FEATURES :base_features,
    IN_TEST_HH    :in_test_hh,
    IP_DAY_TEST_HH:ip_day_test,
    IP_DAY_HH     :ip_day_hh,
    IP_OS_HH      : ip_os_hh,
    IP_APP_HH     :ip_app_hh,
    IP_APP_OS_HH  :ip_app_os_hh,
    APP_DAY_HH    :app_day_hh,
    NON_FRAUD_PERC_IP:get_non_fraud_perc_ip
}

In [11]:
applicable_features=[
    BASE_FEATURES,
    NON_FRAUD_PERC_IP,
    IN_TEST_HH,  
    IP_DAY_TEST_HH,
    IP_DAY_HH     ,
    IP_OS_HH      ,
    IP_APP_HH     ,
    IP_APP_OS_HH  ,
    APP_DAY_HH    
]

In [12]:
def get_features(file):
    df=pd.read_csv(file)
    set_file(file)
    for each_feature in applicable_features:
        print('generating '+each_feature+'...')
        df=feature_config[each_feature](df)
        print(feature_file(each_feature))
    df.to_csv(feature_file('all_features'),index=False)
    print(feature_file('all_features'))
    del df
    import gc
    gc.collect()
    print('done')

In [13]:
get_features(valid_file)
dd=pd.read_csv('eda/features/valid/all_features_valid_hr_is_test_hr.csv')

generating base_features...
deleting  eda/features/valid/base_features_valid_hr_is_test_hr.csv ...
eda/features/valid/base_features_valid_hr_is_test_hr.csv
generating NON_FRAUD_PERC_IP...
deleting  eda/features/valid/perc_ip_valid_hr_is_test_hr.csv ...
eda/features/valid/non_fraud_perc_ip_valid_hr_is_test_hr.csv
generating IN_TEST_HH...
deleting  eda/features/valid/in_test_hh_valid_hr_is_test_hr.csv ...
eda/features/valid/in_test_hh_valid_hr_is_test_hr.csv
generating IP_DAY_TEST_HH...
Aggregating by  ['ip', 'day', 'IN_TEST_HH'] ...
deleting  eda/features/valid/ip_day_test_hh_valid_hr_is_test_hr.csv ...
IP_DAY_TEST_HH max value =  917
eda/features/valid/ip_day_test_hh_valid_hr_is_test_hr.csv
generating IP_DAY_HH...
Aggregating by  ['ip', 'day', 'hour'] ...
deleting  eda/features/valid/ip_day_hh_valid_hr_is_test_hr.csv ...
IP_DAY_HH max value =  237
eda/features/valid/ip_day_hh_valid_hr_is_test_hr.csv
generating IP_OS_HH...
Aggregating by  ['ip', 'day', 'os', 'hour'] ...
deleting  eda/fe

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: perc_ip, dtype: float64

In [14]:
get_features(train_file)
pd.read_csv('eda/features/train/all_features_valid_hr_is_test_hr.csv').head()

generating base_features...
deleting  eda/features/train/base_features_valid_hr_is_test_hr.csv ...
eda/features/train/base_features_valid_hr_is_test_hr.csv
generating NON_FRAUD_PERC_IP...
deleting  eda/features/train/perc_ip_valid_hr_is_test_hr.csv ...
eda/features/train/non_fraud_perc_ip_valid_hr_is_test_hr.csv
generating IN_TEST_HH...
deleting  eda/features/train/in_test_hh_valid_hr_is_test_hr.csv ...
eda/features/train/in_test_hh_valid_hr_is_test_hr.csv
generating IP_DAY_TEST_HH...
Aggregating by  ['ip', 'day', 'IN_TEST_HH'] ...
deleting  eda/features/train/ip_day_test_hh_valid_hr_is_test_hr.csv ...
IP_DAY_TEST_HH max value =  1117
eda/features/train/ip_day_test_hh_valid_hr_is_test_hr.csv
generating IP_DAY_HH...
Aggregating by  ['ip', 'day', 'hour'] ...
deleting  eda/features/train/ip_day_hh_valid_hr_is_test_hr.csv ...
IP_DAY_HH max value =  247
eda/features/train/ip_day_hh_valid_hr_is_test_hr.csv
generating IP_OS_HH...
Aggregating by  ['ip', 'day', 'os', 'hour'] ...
deleting  eda/f

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,click_date,hour,day,minute,perc_ip,IN_TEST_HH,IP_DAY_TEST_HH,IP_DAY_HH,IP_OS_HH,IP_APP_HH,IP_APP_OS_HH,APP_DAY_HH
0,46371,12,1,14,481,2017-11-08 04:15:09,NaN,0,2017-11-08 00:00:00,4,8,15,0.0,1,3,1,1,1,1,2524
1,46680,11,1,19,325,2017-11-08 11:05:23,NaN,0,2017-11-08 00:00:00,11,8,5,0.0,2,6,3,1,1,1,388
2,28564,18,1,13,107,2017-11-06 17:35:13,NaN,0,2017-11-06 00:00:00,17,6,35,0.0,3,3,1,1,1,1,662
3,44527,6,1,15,459,2017-11-08 13:01:18,NaN,0,2017-11-08 00:00:00,13,8,1,0.0,1,29,7,1,1,1,275
4,12505,2,1,25,452,2017-11-08 06:18:29,NaN,0,2017-11-08 00:00:00,6,8,18,0.0,2,11,2,1,2,1,2365


In [14]:
test_file='input/test_parsed.csv'
get_features(test_file)

generating base_features...
deleting  eda/features/test/base_features.csv ...
eda/features/test/base_features.csv
generating IN_TEST_HH...
eda/features/test/in_test_hh.csv
generating IP_DAY_TEST_HH...
Aggregating by  ['ip', 'day', 'IN_TEST_HH'] ...
IP_DAY_TEST_HH max value =  182513
eda/features/test/ip_day_test_hh.csv
generating IP_DAY_HH...
Aggregating by  ['ip', 'day', 'hour'] ...
IP_DAY_HH max value =  40231
eda/features/test/ip_day_hh.csv
generating IP_OS_HH...
Aggregating by  ['ip', 'day', 'os', 'hour'] ...
IP_OS_HH max value =  8224
eda/features/test/ip_os_hh.csv
generating IP_APP_HH...
Aggregating by  ['ip', 'day', 'app', 'hour'] ...
IP_APP_HH max value =  6250
eda/features/test/ip_app_hh.csv
generating IP_APP_OS_HH...
Aggregating by  ['ip', 'day', 'app', 'os', 'hour'] ...
IP_APP_OS_HH max value =  1307
eda/features/test/ip_app_os_hh.csv
generating APP_DAY_HH...
Aggregating by  ['app', 'day', 'hour'] ...
APP_DAY_HH max value =  522993
eda/features/test/app_day_hh.csv
eda/featur

In [ ]:
### TEST TO check whetther correct data is written
# df1=df
# feat_1=[IN_TEST_HH]
# for feat in feat_1:
#     df1=feature_config[feat](df1)
# assert('IN_TEST_HH' in df1.columns)

# print('2nd')
# feat_2=[NON_FRAUD_PERC_IP,IN_TEST_HH]
# for feat in feat_1:
#     df1=feature_config[feat](df1)
# assert('perc_ip' in df1.columns)